### Import packages

In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pathlib import Path
import pandas as pd
import numpy as np
import random
import glob
import os

from add_features import add_xy_future, add_velocity_xy, add_acceleration_xy, add_average_velocity, add_orientation, add_ball_in_motion, add_distance_to_ball, add_angle_to_ball, add_offside, add_distance_to_onside, load_FM_data, add_FM_data, add_tiredness, add_tiredness_short_term
from visualize_game import visualize_game_animation, visualize_prediction_animation
from utils import denominators, google_sheet_to_df, split_match_ids, load_processed_frames, extract_variables, load_tf_model, prepare_EL_input_data, prepare_df, prepare_data, prepare_LSTM_df, prepare_LSTM_input_data, total_error_loss, smooth_predictions_xy, run_model, print_column_variance
from settings import *

2024-04-26 18:06:37.277212: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Necessary columns
| Column Name    | Description                                        |
|----------------|----------------------------------------------------|
| player         | The name of the player                             |
| jersey_number  | The jersey number of the player                    |
| team           | 'home_team', 'away_team', or 'ball'                |
| team_name      | The team of the player                             |
| period         | The period of the game (1 or 2)                    |
| minute         | The minute of the game                             |
| second         | The second within the current minute               |
| frame          | The frame of the game                              |
| distance_ran   | The cumulative distance covered by the player      |
| x              | The x-coordinate of the player                     |
| y              | The y-coordinate of the player                     |

## Define make prediction

In [2]:
# Add all features
def add_all_features(frames_df, match_id):
    # Add the following features
    frames_df = add_xy_future(frames_df, FPS * seconds_into_the_future)
    frames_df = add_velocity_xy(frames_df, 1, smooth=True)
    frames_df = add_acceleration_xy(frames_df, 1, smooth=True)
    frames_df = add_average_velocity(frames_df)
    frames_df = add_orientation(frames_df)
    frames_df = add_ball_in_motion(frames_df)
    frames_df = add_distance_to_ball(frames_df)
    frames_df = add_angle_to_ball(frames_df)
    # frames_df = add_offside(frames_df)
    frames_df = add_distance_to_onside(frames_df)
    frames_df = add_FM_data(frames_df, load_FM_data())
    frames_df = add_tiredness(frames_df)
    frames_df = add_tiredness_short_term(frames_df, window=FPS*20)

    # Add column containing the 'match_id'
    frames_df['match_id'] = match_id

    return frames_df

In [4]:
# Create the vectors 'x_future_pred' and 'y_future_pred' on frames_df
def make_predictions(frames_df, model_name):
    # Prepare the DataFrame by adding all features
    frames_df = add_all_features(frames_df, match_id=1)
    
    # Run the model and add the vectors 'x_future_pred' and 'y_future_pred'
    frames_df = run_model([], model_name, frames_df)

    # Calculate the error
    error = total_error_loss(frames_df)
    print(f"Error: {error} m")

    # Define the file path
    file_path = f"models/{model_name}.txt"
    # if 'Testing results' does not exists in txt file
    with open(file_path, 'r') as file:
        if 'Testing results' not in file.read():
            # Write the following with f.write
            with open(file_path, 'a') as file:  # 'a' mode to append data
                file.write(f"\nTesting results:\ntest_loss: {error}\n")
            print("Testing results added to the file.")

    return frames_df

## Example usage

In [22]:
# Example match
_, test_ids, _ = split_match_ids(n_matches=10)
test_id = test_ids[0]
test_id = 'eaa1aee4-1acf-4a74-acdc-016343b231cc'

# Load the unprocessed version of test_id
file_path = f"{DATA_LOCAL_FOLDER}/data/2023/Allsvenskan/unprocessed/{test_id}.parquet"
frames_df = pd.read_parquet(file_path).iloc[1000:35000]     # Load in 1 minute of unprocessed data

# Make the prediction
model_name = "LSTM_model_v6"
frames_df = make_predictions(frames_df, model_name)

270/270 [==============================] - 1s 927us/step
Error: 2.213 m


In [23]:
visualize_prediction_animation(frames_df, 800, 980, model_name)